In [2]:
import pandas as pd
import json
import ast
import csv
import string
import random

In [1]:
import yaml


def load_settings(file_path):
    with open(file_path, "r") as stream:
        try:
            settings = yaml.safe_load(stream)
            return settings
        except yaml.YAMLError as exc:
            print(f"Error loading YAML file: {exc}")


file_path = "../../../settings.yml"
settings = load_settings(file_path)

In [3]:
SCHEDULE_FILE = "schedule_table.csv"
FLIGHT_FILE = "flight_inventory_table.csv"
PNR_FILE = "pnr_table.csv"
PASSENGER_FILE = "passenger_table.csv"

In [4]:
schedule_df = pd.read_csv(SCHEDULE_FILE) # df_sch
flight_df = pd.read_csv(FLIGHT_FILE) # df_inv
pnr_df = pd.read_csv(PNR_FILE) # df_pnrb
passenger_df = pd.read_csv(PASSENGER_FILE) # df_pnrp

In [5]:
cabins = "FC_CD,BC_CD,PC_CD,EC_CD".split(",")
classes = {}
for k in cabins:
    classes[k] = []

for cabin in cabins:
    for i in range(len(flight_df)):
        classes[cabin] += list(ast.literal_eval(flight_df[cabin][i]).keys())
    classes[cabin] = list(set(classes[cabin]))

classes

{'FC_CD': ['F', 'P'],
 'BC_CD': ['C', 'J', 'Z'],
 'PC_CD': ['R', 'M', 'S', 'Q', 'H', 'T'],
 'EC_CD': ['L',
  'G',
  'K',
  'A',
  'U',
  'N',
  'O',
  'V',
  'I',
  'W',
  'X',
  'Y',
  'D',
  'B',
  'E']}

In [6]:
# populate cabin.csv and class.csv
cabin_df = pd.DataFrame(columns=["abb", "des", "score"])
class_df = pd.DataFrame(columns=["abb", "des", "score", "cabin"])

cabin_data, class_data = [], []

cabins = {
    "F": "First Class",
    "B": "Business Class",
    "P": "Premium Economy",
    "E": "Economy",
}

for cabin in cabins:
    cabin_data.append(
        {
            "abb": cabin,
            "des": cabins[cabin],
            "score": int(settings["scores"]["cabins"][cabin]),
        }
    )
    key = f"{cabin}C_CD"
    for c in classes[key]:
        class_data.append(
            {
                "abb": c,
                "des": f"{cabins[cabin]} - {c}",
                "score": int(settings["scores"]["cabins"][cabin]),
                "cabin": cabin,
            }
        )

cabin_df = pd.concat([cabin_df, pd.DataFrame(cabin_data)]).to_csv(
    "cabin.csv", index=False
)
class_df = pd.concat([class_df, pd.DataFrame(class_data)]).to_csv(
    "class.csv", index=False
)
# print(cabin_df, class_df)
print("Done")

Done


In [7]:
# CarrierCodes
cc_column_name = "CarrierCode"
carrier = []
carrier_df = pd.DataFrame(columns=["code", "desc"])

for i in range(len(flight_df)):
    carrier.append(flight_df[cc_column_name][i])

carrier_data = []
carriers = list(set(carrier))
for carrier in carriers:
    carrier_data.append({"code": carrier, "desc": f"carrier_{carrier}"})

carrier_df = pd.concat([carrier_df, pd.DataFrame(carrier_data)]).to_csv(
    "carrier.csv", index=False
)

In [8]:
settings["scores"]

{'default_ssr_score': 200,
 'default_scd2_score': 300,
 'default_scd3_score': 200,
 'cabins': {'F': 2000, 'B': 1000, 'P': 500, 'E': 100},
 'loyalty': {'Silver': 1000, 'Gold': 2000, 'Platinum': 3000}}

In [9]:
# find ssrs and groups
ssr_column_name, ss_cd1, ss_cd2 = "SSR_CODE_CD1", "SPECIAL_NAME_CD2", "SPECIAL_NAME_CD1"
ssr, s_cd1, s_cd2 = [], [], []

for i in range(len(passenger_df)):
    ssr.append(passenger_df[ssr_column_name][i])
    s_cd1.append(passenger_df[ss_cd1][i])
    s_cd2.append(passenger_df[ss_cd2][i])

ssr, s_cd2, s_cd1 = list(set(ssr)), list(set(s_cd2)), list(set(s_cd1))
# ssr, s_cd2, s_cd1
ssr_df = pd.DataFrame(columns=["abb", "des", "score", "prob"])
ssr_data = []
ssr.pop(0)  # remove nan

for ss in ssr:
    ssr_data.append(
        {
            "abb": ss,
            "des": f"SSR_{ss}",
            "score": int(settings["scores"]["default_ssr_score"]),
            "prob": float(settings["probabilities"]["default_ssr_probability"]),
        }
    )

ssr_df = pd.concat([ssr_df, pd.DataFrame(ssr_data)]).to_csv("ssr.csv", index=False)

/tmp/ipykernel_38040/3262773731.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ssr_df = pd.concat([ssr_df, pd.DataFrame(ssr_data)]).to_csv("ssr.csv", index=False)


In [28]:
# fetch unique aircrafts
tail_numbers, schedule_ids, flight_numbers = [], [], []
aircrafts = []
idx = 1

# schedule_id - index map
schedule_idx = {}
for i in range(len(flight_df)):
    if flight_df["ScheduleId"][i] not in schedule_idx:
        schedule_idx[flight_df["ScheduleId"][i]] = i

for i in range(len(schedule_df)):
    tn = schedule_df["AircraftTailNumber"][i]
    schedule_ids.append(schedule_df["ScheduleID"][i])
    flight_numbers.append(int(schedule_df["FlightNumber"][i]))

    tc, ti = (
        int(flight_df["TotalCapacity"][schedule_idx[schedule_df["ScheduleID"][i]]]),
        flight_df["TotalInventory"][schedule_idx[schedule_df["ScheduleID"][i]]],
    )

    # print sum of FC_CD,BC_CD,PC_CD,EC_CD
    tp = 0
    keys = "FC_CD,BC_CD,PC_CD,EC_CD".split(",")
    for k in keys:
        tp += sum(ast.literal_eval(flight_df[k][schedule_idx[schedule_df["ScheduleID"][i]]]).values())
    # print(tp, tc, ti)

    if tn not in tail_numbers:
        aircrafts.append(
            {
                "id": idx,
                "model": schedule_df["AircraftType"][i],
                "registration": schedule_df["AircraftTailNumber"][i],
                "owner_code": "",
                "owner_name": "",
                "total_capacity": tc,
                "total_inventory": ti,
            }
        )
        idx += 1
        tail_numbers.append(tn)


tail_numbers = list(set(tail_numbers))
tail_numbers, len(tail_numbers)

len(schedule_ids), len(flight_numbers)
# aircrafts
# schedule_ids, flight_numbers are one-to-one mapping hence this is redundant

(90, 90)

In [26]:
print(sum([42,85,127,170]))
f1 = {'F': 25, 'P': 17}
f2 = {'C': 34, 'J': 26, 'Z': 26}
f3 = {'Q': 38, 'R': 25, 'S': 13, 'T': 13, 'H': 25, 'M': 13}
f4 = {'Y': 13, 'A': 13, 'B': 6, 'D': 6, 'E': 6, 'G': 13, 'I': 6, 'K': 13, 'L': 6, 'N': 6, 'O': 6, 'U': 13, 'V': 6, 'W': 6, 'X': 6}

# (42, 86, 127, 125)
sum([sum(f1.values()), sum(f2.values()), sum(f3.values()), sum(f4.values())])

424


380

In [30]:
df_aircrafts = pd.read_csv("aircrafts.csv")
df_seat_distribution = pd.read_csv("seat-distribution.csv")

In [ ]:
df_new_aircrafts = pd.DataFrame(columns=df_aircrafts.columns, index=df_aircrafts.index)
id_aircraft = 0
set_aircrafts = set(zip(schedule_df['AircraftType'], schedule_df['AircraftTailNumber']))
for aircraft_type, tail_number in set_aircrafts:
    df_new_aircrafts.loc[id_aircraft] = [id_aircraft+1, aircraft_type, aircraft_type, tail_number, tail_number, aircraft_type, 0, 0]
    id_aircraft += 1
df_new_aircrafts = df_new_aircrafts.dropna(how='all')

In [ ]:
df_new_seat_distribution = pd.DataFrame(columns=df_seat_distribution.columns, index=range(len(df_seat_distribution)))

id_seat = 0
set_seat_dist = set(zip(flight_df['AircraftType'],flight_df['FC_CD'], flight_df['BC_CD'], flight_df['PC_CD'], flight_df['EC_CD']))
# print(len(set_seat_dist))
for aircraft_type,fc_cd, bc_cd, pc_cd, ec_cd in set_seat_dist:

    fc_cd_dict, bc_cd_dict, pc_cd_dict, ec_cd_dict  = list(map(ast.literal_eval, [fc_cd, bc_cd, pc_cd, ec_cd]))
    total_capacity = 0    
    needed_aircraft_df = df_new_aircrafts[df_new_aircrafts['model'] == aircraft_type]
    
    for fc in fc_cd_dict:
        for aircraft_id in needed_aircraft_df['id']:
            df_new_seat_distribution.loc[id_seat] = (aircraft_id, fc, fc_cd_dict[fc])
            id_seat += 1
        total_capacity += fc_cd_dict[fc]
    for bc in bc_cd_dict:
        for aircraft_id in needed_aircraft_df['id']:
            df_new_seat_distribution.loc[id_seat] = (aircraft_id, bc, bc_cd_dict[bc])
            id_seat += 1
        total_capacity += bc_cd_dict[bc]
    for pc in pc_cd_dict:
        for aircraft_id in needed_aircraft_df['id']:
            df_new_seat_distribution.loc[id_seat] = (aircraft_id, pc, pc_cd_dict[pc])
            id_seat += 1
        total_capacity += pc_cd_dict[pc]
    for ec in ec_cd_dict:
        for aircraft_id in needed_aircraft_df['id']:
            df_new_seat_distribution.loc[id_seat] = (aircraft_id, ec, ec_cd_dict[ec])
            id_seat += 1
        total_capacity += ec_cd_dict[ec]

    
    for index, row in needed_aircraft_df.iterrows():
        row.total_capacity = total_capacity
        row.total_inventory = total_capacity
        df_new_aircrafts.loc[index] = row
        
    
    
df_new_seat_distribution = df_new_seat_distribution.dropna(how='all')

In [ ]:
df_new_aircrafts.to_csv('aircrafts.csv', index=False)
df_new_seat_distribution.to_csv('seat-distribution.csv', index=False)